In [ ]:
! pip install -U accelerate
! pip install -U transformers
! pip install wandb
!pip install datasets
!pip install evaluate

from tqdm import tqdm, trange
import multiprocessing
from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,
                          RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)
from datasets import Dataset
from transformers import RobertaTokenizer, RobertaForMaskedLM, pipeline
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

import torch
import os

import evaluate
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import accelerate
import transformers


transformers.__version__, accelerate.__version__

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 16.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is 

('4.37.2', '0.26.1')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.listdir("/content/drive/My Drive/LSEG_Dataset")

file_path = "/content/drive/My Drive/LSEG_Dataset/cwe476_Data.csv"
df = pd.read_csv(file_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.head()


,code,CWE-476,Label
0,"Perl_mro_get_linear_isa(pTHX_ HV *pTHX_PointerMarker)\n{\n struct mro_meta* structPointerMarker;\n AV *AVPointerMarker;\n\n PERL_ARGS_ASSERT_MRO_GET_LINEAR_ISA;\n if(!SvOOK(pTHX_PointerMarker))\n Perl_croak(aTHX_ ""Can't linearize anonymous symbol table"");\n\n structPointerMarker = HvMROMETA(pTHX_PointerMarker);\n if (!structPointerMarker->mro_which)\n Perl_croa...",True,1
1,handlePRERET(FlowInstruction *FlowInstructionPointerMarker)\n{\n BasicBlock *BasicBlockPointerMarker = FlowInstructionPointerMarker->bb;\n BasicBlock *BasicBlockPointerMarker = FlowInstructionPointerMarker->target.bb;\n\n FlowInstructionPointerMarker->subOp = NV50_IR_SUBOP_EMU_PRERET + 0;\n BasicBlockPointerMarker->remove(FlowInstructionPointerMarker);\n BasicBlockPointerMarker->ins...,True,1
2,"GetInetAddr(char *charPointerMarker)\n\n{ struct StructMarker StructVariable;\n struct hostent *structPointerMarker;\n\nStructVariable.s_addr = inet_addr(charPointerMarker);\n\nif ((StructVariable.s_addr == INADDR_NONE) || (StructVariable.s_addr == 0)) \n {\n if ((structPointerMarker = gethostbyname(charPointerMarker)) == 0)\n {\n snprintf(OUTPUT,CF_BUFSIZE,""\nhost not found:...",True,1
3,"do_redirect(action_list_t *action_list_tPointerMarker, const char *charPointerMarker, int intMarker)\n{\n action_list_t *action_list_tPointerMarker = NULL;\n\n \n\n \n while (action_list_tPointerMarker != NULL) {\n action_list_tPointerMarker = action_list_tPointerMarker;\n if (action_list_tPointerMarker->action_list_tPointerMarker == ACTION_REJECT)\n return SIEVE_RUN_ERROR;\n ...",True,1
4,"gf_af_new(GF_Compositor *GF_CompositorPointerMarker, GF_AudioInterface *GF_AudioInterfacePointerMarker, char *charPointerMarker)\n{\n GF_AudioFilterItem *GF_AudioFilterItemPointerMarker;\n if (!GF_AudioInterfacePointerMarker || !charPointerMarker) return NULL;\n\n GF_SAFEALLOC(GF_AudioFilterItemPointerMarker, GF_AudioFilterItem);\n\n GF_AudioFilterItemPointerMarker->GF_AudioInterfacePointerMa...",True,1


In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
dataset = {}
dataset['text'] = list(df_train['code'])
dataset['label'] = list(df_train['Label'])
ds = Dataset.from_dict(dataset)
ds = ds.train_test_split(test_size=0.1)

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base-mlm')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_dataset = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/690497 [00:00<?, ? examples/s]

Map:   0%|          | 0/76722 [00:00<?, ? examples/s]

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/codebert-base-mlm")

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base-mlm and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
training_args = TrainingArguments(
    output_dir="trained_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
